In [ ]:
!pip install gradio groq  numpy
!pip install openai-whisper
!pip install librosa
!pip install gtts



In [8]:
import os
import gradio as gr
import whisper
from groq import Groq
import numpy as np
import librosa

# Set up Groq API client
client = Groq(
    api_key="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",
)

# Load Whisper model
model = whisper.load_model("base")

def chatbot(audio):
    try:
        # Check if the audio data is None
        if audio is None:
            return "Error: No audio input provided."

        # Load audio file using librosa to ensure correct format
        audio_data, sample_rate = librosa.load(audio, sr=16000)  # Whisper typically expects 16kHz sample rate

        # Ensure audio data is in float32 format
        if not np.issubdtype(audio_data.dtype, np.floating):
            audio_data = audio_data.astype(np.float32)

        # Transcribe the audio input using Whisper
        transcription = model.transcribe(audio_data)
        user_input = transcription["text"]

        # Generate feedback using Llama3 70B via Groq API
        messages = [
            {"role": "system", "content": "You are an English tutor. Your job is to provide detailed feedback on grammar and vocabulary."},
            {"role": "user", "content": f"Please review the following text and provide feedback on grammar and vocabulary: {user_input}"}
        ]

        chat_completion = client.chat.completions.create(
            messages=messages,
            model="llama3-groq-70b-8192-tool-use-preview",
        )
        response_text = chat_completion.choices[0].message.content

        return response_text
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom interface
def build_interface():
    with gr.Blocks() as demo:
        gr.Markdown(
            """
            <h1 style="text-align: center; color: #4CAF50;">Voice-to-Text Chatbot</h1>
            <h3 style="text-align: center;">Powered by OpenAI Whisper and Llama3 70B</h3>
            <p style="text-align: center;">Speak to the chatbot and get detailed feedback on your grammar and vocabulary in real-time. Start by recording your voice.</p>
            """
        )
        with gr.Row():
            with gr.Column(scale=1):
                audio_input = gr.Audio(type="filepath", label="Record Your Voice")
            with gr.Column(scale=2):
                chatbot_output_text = gr.Textbox(label="Feedback", lines=5)

        # Automatically process the recording when it stops
        audio_input.change(chatbot, inputs=audio_input, outputs=chatbot_output_text)

    return demo

# Launch the interface
if __name__ == "__main__":
    interface = build_interface()
    interface.launch()


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 74.0MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://87ad61485099901fa3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [9]:
import gradio as gr
from gtts import gTTS
import os

def text_to_speech(text):
    # Convert the input text to speech using gTTS
    tts = gTTS(text=text, lang='en')
    audio_file_path = "output.mp3"
    tts.save(audio_file_path)

    # Return the path to the audio file for Gradio
    return audio_file_path

# Gradio UI
def build_interface():
    with gr.Blocks() as demo:
        gr.Markdown(
            """
            <h1 style="text-align: center; color: #4CAF50;">Text to Speech Converter</h1>
            <p style="text-align: center;">Type a sentence, and the AI will pronounce it for you. Simply enter the text and listen to the audio output.</p>
            """
        )
        with gr.Row():
            text_input = gr.Textbox(label="Enter your text here", lines=3, placeholder="Type your text here...")
            convert_button = gr.Button("Convert to Speech")
            audio_output = gr.Audio(label="Audio Output")

        convert_button.click(text_to_speech, inputs=text_input, outputs=audio_output)

    return demo

if __name__ == "__main__":
    interface = build_interface()
    interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1375db1c16e4d64aef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [11]:
!pip install gradio
!pip install nltk
!pip install deep-translator

In [12]:
import gradio as gr
from nltk.corpus import wordnet
from deep_translator import GoogleTranslator

def get_synonyms(word):
    try:
        synonyms = wordnet.synsets(word)
        english_synonyms = set()
        translator = GoogleTranslator(source='en', target='ur')

        for syn in synonyms:
            for lemma in syn.lemmas():
                english_synonyms.add(lemma.name())

        # Translate synonyms to Urdu
        urdu_synonyms = [translator.translate(word) for word in english_synonyms]

        return f"English Synonyms: {', '.join(english_synonyms)}", f"Urdu Synonyms: {', '.join(urdu_synonyms)}"
    except Exception as e:
        return f"An error occurred: {e}", ""

# Gradio Interface
iface = gr.Interface(
    fn=get_synonyms,
    inputs="text",
    outputs=["text", "text"],
    title="Synonym Finder",
    description="Enter a word to get its English and Urdu synonyms."
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://46bf695eda87ee4dc9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
